# Train the Classifier Model

The last step in the original flow was the training of the classifier step. We'll again use a [`Tensorflow` SageMaker SDK estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html#train-a-model-with-tensorflow) to create the model. As before, we'll set up some variables to point the estimator to the data and give it the hyperparameters for training.

In [7]:
bucket = "sagemaker-us-east-1-160951647621"
data_path = "data-augmentation-2020-10-05-21-59-11-375/output/classifier/train"

inputs = {
    "train": f"s3://{bucket}/{data_path}"
}
print(inputs)

{'train': 's3://sagemaker-us-east-1-160951647621/data-augmentation-2020-10-05-21-59-11-375/output/classifier/train'}


In [2]:
train_instance_type = "ml.p3.2xlarge"
hyperparameters = {
    #"max-rows": 1024,
    "num-epochs": 10,
    "batch-size": 1024,
}

## Training Script

In [5]:
!pygmentize src/train_classif.py

import numpy as np
import logging
import argparse
import os
from pathlib import Path
from tensorflow.keras import layers, Input, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


def parse_arguments():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--num-epochs", type=int, default=15)
    parser.add_argument("--batch-size", type=int, default=1024)
    parser.add_argument("--model_dir", type=str, default=None)
    parser.add_argument('--model-save-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    return parser.parse_args()


def create_classifier(x_train, x_test, y_train, y_test, epochs, batch_size):
    de

## Execute Training Job

With the script and data ready, we can prepare and execute the training.

In [8]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='src/train_classif.py',
                       base_job_name='train-classifier',
                       instance_type=train_instance_type,
                       instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(), # Passes to the container the AWS role that you are using on this notebook
                       framework_version='2.1.0',
                       py_version='py3')

In [9]:
estimator.fit(inputs)

2020-10-05 22:13:18 Starting - Starting the training job...
2020-10-05 22:13:20 Starting - Launching requested ML instances......
2020-10-05 22:14:26 Starting - Preparing the instances for training......
2020-10-05 22:15:49 Downloading - Downloading input data
2020-10-05 22:15:49 Training - Downloading the training image.........
2020-10-05 22:17:10 Training - Training image download completed. Training in progress..2020-10-05 22:17:14,949 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-10-05 22:17:15,433 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch-size": 1024,
        "num-epochs": 10,
        "model_dir": "s3://sagem

In [10]:
print(f"The model is available at {estimator.model_data}")

The model is available at s3://sagemaker-us-east-1-160951647621/train-classifier-2020-10-05-22-13-17-852/output/model.tar.gz


## Deploying the Model

Deploying the model directly is just one command. The deployment itself takes a few minutes to create the instance serving it.

In [69]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c5.2xlarge')

-------------!

## Calling Inference

To call inference on the endpoint, we first download the data we have.

In [13]:
sagemaker.utils.download_file(bucket, data_path + "/data.npz", "/tmp/data.npz", sagemaker.session.Session())

In [14]:
import numpy as np

with np.load("/tmp/data.npz", allow_pickle=True) as data:
    x = data['x']
    y = data['y']
    label_classes = data['label_classes'].item(0)
prediction_translator =dict(map(reversed, label_classes.items()))

Then we build a function that will call inference and compare it to the actual value:

In [55]:
def predict_compare(x, y, predictor, translator=prediction_translator):
    payload = {
        "instances": [x.tolist()]
    }
    predictions = predictor.predict(payload)['predictions']
    predicted_class = prediction_translator[np.array(predictions).argmax()]
    return predicted_class, y

Finally, we take a subset with only the failures to evaluate how well our model is doing.

In [58]:
failed_x = x[y!='none']
failed_y = y[y!='none']

In [71]:
correct_predict = 0
for i in np.random.randint(failed_y.shape[0], size=100):
    predicted, actual = predict_compare(failed_x[i], failed_y[i], predictor)
    correct_predict += predicted == actual
    print(f"{i:5d}: Correct: {predicted==actual}\t\tActual x Predicted: {actual} x {predicted}")
    
print(f"Correct: {correct_predict} out of 100")

15903: Correct: True		Actual x Predicted: Random x Random
15017: Correct: True		Actual x Predicted: Random x Random
 7994: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
  887: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
  856: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
 5415: Correct: True		Actual x Predicted: Center x Center
 8102: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
  568: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
10244: Correct: True		Actual x Predicted: Edge-Ring x Edge-Ring
12934: Correct: True		Actual x Predicted: Near-full x Near-full
19369: Correct: True		Actual x Predicted: Donut x Donut
18198: Correct: True		Actual x Predicted: Scratch x Scratch
 7663: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
 7159: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
 2450: Correct: True		Actual x Predicted: Edge-Loc x Edge-Loc
 3260: Correct: True		Actual x Predicted: Center x Center
11226: Correct: True		Actual

In general, we're getting at least 95% accuracy on the results. This was a simple model on a simple dataset, very different results can be achieved depending on your model and data.

To finish things, remember to delete the endpoint to avoid incurring in more costs.

In [66]:
predictor.delete_endpoint()